In [1]:
import sys
import os
import numpy as np
import pandas as pd
import re
import collections
from collections import defaultdict
from scipy import stats


In [2]:
df = pd.read_csv('C:/Users/reena/Desktop/recommender/Original_data.csv')
df.shape

C:\Users\reena\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(605715, 19)

In [3]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0,Columbia University,Computer Science,MS,NaN,Wait listed,E-mail,"(24, 6, 2021)",24 Jun 2021,NaN,NaN,NaN,NaN,NaN,NaN,International,"(24, 6, 2021)",24 Jun 2021,Lol waitlisted but I seriously think is someon...
1,1,Georgetown University,Global Health,MS,NaN,Accepted,E-mail,"(5, 3, 2021)",05 Mar 2021,3.7,NaN,NaN,NaN,NaN,NaN,American,"(24, 6, 2021)",24 Jun 2021,Got an email to check my portal. I studied in ...
2,2,University Of California Irvine,INP,PhD,NaN,Accepted,Phone,"(9, 6, 2021)",09 Jun 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(24, 6, 2021)",24 Jun 2021,reply
3,3,Columbia University,Computer Science,MS,NaN,Wait listed,E-mail,"(24, 6, 2021)",24 Jun 2021,NaN,165.0,167.0,NaN,True,NaN,American,"(24, 6, 2021)",24 Jun 2021,Waitlisted today after writing multiple strong...
4,4,Columbia University,Computer Science,MS,NaN,Wait listed,E-mail,"(24, 6, 2021)",24 Jun 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(24, 6, 2021)",24 Jun 2021,TTPB: I too received a rejection today. But di...


In [4]:
df.drop(['Unnamed: 0','3','6','5','7','17','15','16','13'],axis = 1, inplace = True)
df.head()

,0,1,2,4,8,9,10,11,12,14
0,Columbia University,Computer Science,MS,Wait listed,NaN,NaN,NaN,NaN,NaN,International
1,Georgetown University,Global Health,MS,Accepted,3.7,NaN,NaN,NaN,NaN,American
2,University Of California Irvine,INP,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,NaN
3,Columbia University,Computer Science,MS,Wait listed,NaN,165.0,167.0,NaN,True,American
4,Columbia University,Computer Science,MS,Wait listed,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns = ['University', 'Major', 'Program', 'Decision', 'Cumulative_gpa', 'GRE_Verbal', 'GRE_Quant', 
           'GRE_AWA', 'Retake','Status']
df.head()

,University,Major,Program,Decision,Cumulative_gpa,GRE_Verbal,GRE_Quant,GRE_AWA,Retake,Status
0,Columbia University,Computer Science,MS,Wait listed,NaN,NaN,NaN,NaN,NaN,International
1,Georgetown University,Global Health,MS,Accepted,3.7,NaN,NaN,NaN,NaN,American
2,University Of California Irvine,INP,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,NaN
3,Columbia University,Computer Science,MS,Wait listed,NaN,165.0,167.0,NaN,True,American
4,Columbia University,Computer Science,MS,Wait listed,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = df[df['Decision'] =='Accepted']

df.shape

(239065, 10)

In [7]:
df.isnull().sum()

University             0
Major                  2
Program                0
Decision               0
Cumulative_gpa    153982
GRE_Verbal        167510
GRE_Quant         167510
GRE_AWA           170119
Retake            167510
Status             23640
dtype: int64

In [8]:
df1 = pd.DataFrame(df)
df1.drop_duplicates(['University'])
df1

,University,Major,Program,Decision,Cumulative_gpa,GRE_Verbal,GRE_Quant,GRE_AWA,Retake,Status
1,Georgetown University,Global Health,MS,Accepted,3.7,NaN,NaN,NaN,NaN,American
2,University Of California Irvine,INP,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,NaN
5,University Of Gothenburg,Linguistics,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,International
11,Texas A&M University - College Station (TAMU),Electrical And Computer Engineering,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,International
12,University Of Western Ontario,Pathology And Laboratory Medicine,MS,Accepted,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
605708,"Indiana University, Bloomington",Telecommunications,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,NaN
605709,"University Of Wisconsin, Madison",Comparative Biomedical Sciences,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,NaN
605711,Purdue University,Nuclear Engineering,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,NaN
605712,Michigan State University,Communication,PhD,Accepted,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
import string
X=df['University'].str.partition('University')
 

In [29]:
X

,0,1,2
1,Georgetown,University,
2,,University,Of California Irvine
5,,University,Of Gothenburg
11,Texas A&M,University,- College Station (TAMU)
12,,University,Of Western Ontario
...,...,...,...
605708,Indiana,University,", Bloomington"
605709,,University,"Of Wisconsin, Madison"
605711,Purdue,University,
605712,Michigan State,University,


In [62]:
mask = X[2].str.contains('of', case=False).fillna(False)

X['23'] = X.loc[mask, 1] + X[2]
X['23']

1                                      NaN
2          University Of California Irvine
5                 University Of Gothenburg
11                                     NaN
12           University Of Western Ontario
                        ...               
605708                                 NaN
605709    University Of Wisconsin, Madison
605711                                 NaN
605712                                 NaN
605713                                 NaN
Name: 23, Length: 239065, dtype: object

In [72]:
mask = X[0].str.contains('', case=False).fillna(True)

X['12'] = X.loc[mask,0] + X[1]
X['12']

1             Georgetown University
2                        University
5                        University
11             Texas A&M University
12                       University
                    ...            
605708           Indiana University
605709                   University
605711            Purdue University
605712    Michigan State University
605713        Ohio State University
Name: 12, Length: 239065, dtype: object

In [85]:
X['23'] = X['23'].replace(np.nan, '', regex=True)
X

,0,1,2,23,12,University
1,Georgetown,University,,,Georgetown University,NaN
2,,University,Of California Irvine,University Of California Irvine,University,UniversityUniversity Of California Irvine
5,,University,Of Gothenburg,University Of Gothenburg,University,UniversityUniversity Of Gothenburg
11,Texas A&M,University,- College Station (TAMU),,Texas A&M University,NaN
12,,University,Of Western Ontario,University Of Western Ontario,University,UniversityUniversity Of Western Ontario
...,...,...,...,...,...,...
605708,Indiana,University,", Bloomington",,Indiana University,NaN
605709,,University,"Of Wisconsin, Madison","University Of Wisconsin, Madison",University,"UniversityUniversity Of Wisconsin, Madison"
605711,Purdue,University,,,Purdue University,NaN
605712,Michigan State,University,,,Michigan State University,NaN


In [95]:
if X['12']=='University\n':
    X['12'].replace('University','')
X['12']   

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [37]:
df.dropna()

,University,Major,Program,Decision,Cumulative_gpa,GRE_Verbal,GRE_Quant,GRE_AWA,Retake,Status
67,University Of Florida,Information Systems And Operations Management,MS,Accepted,3.47,145.0,167.0,3.0,True,American
86,New York University,Mathematics,MS,Accepted,3.57,167.0,168.0,4.0,True,American
115,Claremont Graduate University,Mathematics,PhD,Accepted,2.76,163.0,159.0,4.0,True,American
136,Florida State University (FSU),Public Administration,PhD,Accepted,3.41,154.0,168.0,3.5,True,International
142,University Of Alabama,School Psychology,PhD,Accepted,3.97,150.0,140.0,4.5,True,American
...,...,...,...,...,...,...,...,...,...,...
555247,Clemson University,Electrical & Computer Engineering,PhD,Accepted,3.30,470.0,790.0,3.5,False,International
555255,The Rockefeller University,David Rockefeller Graduate Program,PhD,Accepted,3.96,750.0,760.0,5.5,False,American
555273,University Of Chicago,Art Historyganic Chemistry,PhD,Accepted,3.30,510.0,770.0,4.0,False,American
555299,University Of Chicago,Chemistry,PhD,Accepted,3.86,620.0,800.0,4.5,False,International with US Degree
